# Interactive t-SNE ROOTS dimensionality reduction

In [1]:
import pandas as pd
import datashader as ds
import colorcet as cc

import holoviews as hv
from holoviews.operation.datashader import datashade, rasterize, dynspread
import panel as pn

pn.extension()
hv.extension('bokeh')

/home/runner/work/examples/examples/t_sne_roots/envs/default/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "



*Visualization of t-Distributed Stochastic Neighbor Embedding of ROOTS language Corpus.*

Trained data from Christopher Akiki (dataset `cakiki/roots-tsne-data` on Hugging Face 

## Load the data

In [2]:
tsne_embedding = pd.read_parquet('./data/train-00000-of-00001-9a434d9cf7fd233e.parquet')
df = pd.DataFrame(data=tsne_embedding, columns=['x','y','language'])
df.head()

,x,y,language
0,-18.282380,-2.315064,Marathi
1,-64.574786,-6.972784,Marathi
2,-55.588644,0.740308,Marathi
3,30.174092,28.561195,Marathi
4,-16.844400,-0.429286,Marathi


## Datashade and add hover layer

Note that until client-side colormixing is supported, the `datashade` operation is used with an invisible hover layer overlaid on top.

In [3]:
points = hv.Points(df, ['x','y'], ['language'])
op = datashade(points, color_key=cc.glasbey_light,
               aggregator=ds.by('language', ds.count())).opts(show_legend=False, bgcolor='black', frame_width=600, aspect=1)
hover_layer = rasterize(points, selector=ds.first("x"), x_sampling=2, y_sampling=2).opts(tools=["hover"], alpha=0)

### Render

In [4]:
dynspread(op) * hover_layer

:DynamicMap   []
   :Overlay
      .RGB.I   :RGB   [x,y]   (R,G,B,A)
      .Image.I :Image   [x,y]   (x_y Count,language)

### Build a small dashboard with panel

In [5]:
text = """
### Visualization of t-Distributed Stochastic Neighbor Embedding of ROOTS language Corpus. <br> Trained data from Christopher Akiki (dataset `cakiki/roots-tsne-data` on Hugging Face
"""

template = pn.template.FastListTemplate(
    title="t-SNE clustering of ROOTS language corpus"
)

template.main.append(
    pn.Column(
        pn.pane.Markdown(text),
        pn.Row(pn.HSpacer(), dynspread(op) * hover_layer, pn.HSpacer())
    )
)
template.servable();